In [1]:
from models.coreset import RandomCoreset
from models.vanilla_mlp import VanillaMLP
from models.vcl import VCLModel
from datasets import PermutedMnist
import matplotlib.pyplot as plt

import numpy as np

import os

import torch
import torch.nn.functional as F

%load_ext autoreload
%autoreload 2

In [60]:
num_epochs = 100
lr = 1e-3

num_samples = 10
NO_RUNS = 3

random_seed = 1
model_type = 'mlp'
torch.manual_seed(random_seed + 1)
np.random.seed(random_seed)

model_base_name = 'coreset_only_model_type_{}_lr_{}_batch_{}_coresetsize_{}_epochs_{}_run_{}'

In [61]:
batch_size = 256
dataset = PermutedMnist(10)
random_coreset = RandomCoreset(dataset)

_, test_loaders = dataset.get_tasks()

coreset_sizes = [200, 400, 1000, 2500, 5000]
for coreset_size in coreset_sizes:
    for run_number in range(NO_RUNS):
        coreset_loaders,_ = random_coreset.get_coreset_loaders(batch_size=coreset_size, coreset_size=coreset_size)

        model = VanillaMLP(784, 10, [100, 100])
        
        no_tasks = len(coreset_loaders)
        accuracies = np.zeros((no_tasks, no_tasks))
        
        model_name = model_base_name.format(model_type, lr, batch_size, coreset_size, num_epochs, run_number) 
        
        LOG_DIR = 'logs/coreset_only'
        os.makedirs(LOG_DIR, exist_ok=True)
        
        LOG_FILE_NAME = os.path.join(LOG_DIR, model_name) + '.txt'
        log_file_handler = open(LOG_FILE_NAME, "w")
        print("Run Number: {}. Log File: {}".format(run_number, LOG_FILE_NAME))
        
        for i in range(no_tasks):
            train_loader = coreset_loaders[i]
        
            model.train_model(num_epochs, train_loader, lr)
            task_accs = []
            for j in range(i + 1):
                test_loader =  test_loaders[j]
                accuracy = model.get_accuracy(test_loader)
                
                msg = "[Number Tasks: {}]: Task {} Accuracy: {}".format(j + 1, j + 1, accuracy ) 
                log_file_handler.write(msg + '\n')
                log_file_handler.flush()
                print(msg)
                
                task_accs.append(accuracy)
                accuracies[i][j] = accuracy
            msg = "Task {} / {}. Mean Accuracy: {}".format(i + 1, no_tasks, np.mean(task_accs))
            log_file_handler.write(msg + '\n')
            log_file_handler.flush()
            print(msg)

Run Number: 0. Log File: logs/coreset_only/coreset_only_model_type_mlp_lr_0.001_batch_256_coresetsize_200_epochs_100_run_0.txt


Loss: 0.0016345460899174213: 100%|██████████| 100/100 [00:15<00:00,  6.64it/s]


[Number Tasks: 1]: Task 1 Accuracy: 0.7922
Task 1 / 10. Mean Accuracy: 0.7922


Loss: 0.0006022578454576433: 100%|██████████| 100/100 [00:15<00:00,  6.32it/s]


[Number Tasks: 1]: Task 1 Accuracy: 0.8002
[Number Tasks: 2]: Task 2 Accuracy: 0.8038
Task 2 / 10. Mean Accuracy: 0.802


...

[Number Tasks: 1]: Task 1 Accuracy: 0.9197
[Number Tasks: 2]: Task 2 Accuracy: 0.9279
[Number Tasks: 3]: Task 3 Accuracy: 0.9291
[Number Tasks: 4]: Task 4 Accuracy: 0.9288
[Number Tasks: 5]: Task 5 Accuracy: 0.9302
[Number Tasks: 6]: Task 6 Accuracy: 0.9244
[Number Tasks: 7]: Task 7 Accuracy: 0.9299
[Number Tasks: 8]: Task 8 Accuracy: 0.924
Task 8 / 10. Mean Accuracy: 0.92675


Loss: 0.0003415554367368006: 100%|██████████| 100/100 [07:18<00:00,  4.39s/it]


[Number Tasks: 1]: Task 1 Accuracy: 0.9209
[Number Tasks: 2]: Task 2 Accuracy: 0.9269
[Number Tasks: 3]: Task 3 Accuracy: 0.9257
[Number Tasks: 4]: Task 4 Accuracy: 0.9283
[Number Tasks: 5]: Task 5 Accuracy: 0.9306
[Number Tasks: 6]: Task 6 Accuracy: 0.9223
[Number Tasks: 7]: Task 7 Accuracy: 0.9281
[Number Tasks: 8]: Task 8 Accuracy: 0.9251
[Number Tasks: 9]: Task 9 Accuracy: 0.9276
Task 9 / 10. Mean Accuracy: 0.9261666666666666


Loss: 0.000319579170900397: 100%|██████████| 100/100 [07:09<00:00,  4.30s/it] 


[Number Tasks: 1]: Task 1 Accuracy: 0.919
[Number Tasks: 2]: Task 2 Accuracy: 0.9242
[Number Tasks: 3]: Task 3 Accuracy: 0.9269
[Number Tasks: 4]: Task 4 Accuracy: 0.9247
[Number Tasks: 5]: Task 5 Accuracy: 0.9274
[Number Tasks: 6]: Task 6 Accuracy: 0.92
[Number Tasks: 7]: Task 7 Accuracy: 0.9275
[Number Tasks: 8]: Task 8 Accuracy: 0.9273
[Number Tasks: 9]: Task 9 Accuracy: 0.9284
[Number Tasks: 10]: Task 10 Accuracy: 0.9221
Task 10 / 10. Mean Accuracy: 0.9247500000000001
